In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from unidecode import unidecode

%matplotlib notebook

In [ ]:
df = pd.read_csv('../data/preprocessed/details_home_for_sale_dataset_2019-07-25.csv')
df.head(5)


In [ ]:
columns_to_analyze = [
    'ano_de_construccion', 'aptos_por_piso', 'banos', 'banos_extra', 'cantidad_de_pisos',
    'description', 'disposicion', 'distancia_al_mar', 'dormitorios', 'dormitorios_extra',
    'estado', 'extra', 'garajes', 'garajes_extra', 'gastos_comunes', 'gastos_comunes_cur',
    'kind', 'm2_de_la_terraza', 'm2_del_terreno', 'm2_edificados', 'oficina', 'penthouse',
    'piso', 'plantas', 'plantas_extra', 'price', 'price_cur', 'sobre', 'tipo_propiedad',
    'vista_al_mar', 'vivienda_social', 'zona', 
]

In [ ]:
df_obj = (df[columns_to_analyze]
          .select_dtypes(include='object')
          .fillna('')
          .apply(lambda x: x.str.lower().apply(lambda y: unidecode(y)))
         )
df_num = df.select_dtypes(exclude='object')
df_raw = pd.concat([df_num, df_obj], axis=1)

In [ ]:
mask = (
    df_raw['tipo_propiedad'].isin(['apartamentos', 'casas'])
    & (df_raw['oficina'] != 1)
    & (df_raw['penthouse'] != 1) 
)
df_raw = df_raw[mask]
df_raw.columns

In [ ]:
columns = ['ano_de_construccion', 'banos', 'disposicion', 'dormitorios',
           'estado', 'garajes', 'gastos_comunes',
           'm2_de_la_terraza', 'm2_del_terreno', 'm2_edificados',
           'price', 'tipo_propiedad', 'vivienda_social', 'zona']

In [ ]:
df_selected = df_raw[columns].reset_index(drop=True)
fill_zero_col = ['m2_de_la_terraza', 'vivienda_social', 'gastos_comunes', 'garajes']
df_selected.loc[:, fill_zero_col] = df_selected.loc[:, fill_zero_col].fillna(0)
mask_m2_terreno = df_selected['m2_del_terreno'].isna()
df_selected.loc[mask_m2_terreno, 'm2_del_terreno'] = df_selected.loc[mask_m2_terreno, 'm2_edificados']
(
    df_selected
    .isna()
    .sum()
)

In [ ]:
df_selected.dropna(inplace=True)

In [ ]:
df_num_selected = df_selected.select_dtypes(exclude='object')
df_obj_selected = df_selected.select_dtypes(include='object')

In [ ]:
df_obj_selected.sample(5)

In [ ]:
zone_codes = pd.get_dummies(df_obj_selected['zona'])
property_type = pd.get_dummies(df_obj_selected['tipo_propiedad'])
property_state = pd.get_dummies(df_obj_selected['estado'])

In [ ]:
zone_codes.columns = ['ZN_{}'.format(k.replace(' ', '_')) for k in zone_codes.columns]
property_state.columns = ['ST_{}'.format(k.replace(' ', '_')) for k in property_state.columns]


In [ ]:
features_selected = pd.concat([df_num_selected, zone_codes, property_type, property_state], axis=1)
features_selected.to_csv('../data/ready/dataset_houses.csv', index=False)

In [ ]:
price =  features_selected['price']
features = features_selected.drop(columns=['price'])



# Modeling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import Normalizer

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import TSNE
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# scaler =  RobustScaler(quantile_range=(25, 75))
scaler = QuantileTransformer(output_distribution='normal') 
# scaler = PowerTransformer(method='box-cox')  # Non Linear | Strictly positive data
# scaler = PowerTransformer(method='yeo-johnson')  # Non Linear 



In [ ]:
train_features, test_features = train_test_split(features, test_size=0.2, random_state=14)
train_price, test_price = train_test_split(price, test_size=0.2, random_state=14)

scaler.fit(train_features)
sca_train = scaler.transform(train_features)
sca_test = scaler.transform(train_features)

In [ ]:
pca = PCA(n_components=.95, )
pca.fit(sca_train)
arr_train = pca.transform(sca_train)
arr_test = pca.transform(sca_test)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
sns.pairplot(pd.DataFrame(arr[:, 0:4]))

In [ ]:
# color = features.loc[features.index, 'zona'].astype('category').cat.codes
color = train_features.iloc[:, 1]
f, ax = plt.subplots(2,2)
ax[0, 0].scatter(arr[:,0], arr[:,1], c=color)
ax[0, 1].scatter(arr[:,1], arr[:,2], c=color)
ax[1, 0].scatter(arr[:,0], arr[:,2], c=color)
ax[1, 1].scatter(arr[:,3], arr[:,5], c=color)

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=3).fit_transform(scaler.fit_transform(train_features),
                                                              train_price)

In [ ]:
color = train_features.iloc[:, 1]

f, ax = plt.subplots(2, 2)
ax[0, 0].scatter(lda[:, 0], lda[:, 1], c=color)
ax[0, 1].scatter(lda[:, 0], lda[:, 2], c=color)
ax[1, 0].scatter(lda[:, 1], lda[:, 2], c=color)


In [ ]:
def plot_predict_result(test_price, predict_price):
    f, ax = plt.subplots(2)
    ax[0].scatter(test_price, predict_price)
    ax[0].plot([0, 1600000], [0, 1600000], c='r')
    ax[1].hist(test_price - predict_price, bins=100)
    return f, ax

def get_scores(test_train, test_predict):
    mse = mean_squared_error(test_train, test_predict)
    mea = mean_absolute_error(test_train, test_predict)
    r2 = r2_score(test_train, test_predict)
    return 'mse={}, mea={}, r2={}'.format(mse, mea, r2)

# Linear Model

In [ ]:
model = LinearRegression(fit_intercept=True)
model.fit(arr_train, train_price)
#model.coef_, model.intercept_, model.rank_

In [ ]:
predict_price = model.predict(arr_test)
get_scores(test_price, predict_price)

In [ ]:
f, ax = plot_predict_result(test_price, predict_price)
ax[0].set(ylim=[-10000000000, 10000000000])

# Regression Tree

In [ ]:
cls_tree = DecisionTreeRegressor()
cls_tree.fit(train_features, train_price)


In [ ]:
predic_price_tree = cls_tree.predict(test_features)
get_scores(predic_price_tree, predict_price)

In [ ]:
plot_predict_result(test_price, predic_price_tree)

In [ ]:
f, ax = plt.subplots()
(pd
 .Series(cls_tree.feature_importances_, index=train_features.columns)
 .sort_values()[-10:]
 .plot(ax=ax, kind='barh')
)

In [ ]:
df_raw[df_raw['gastos_comunes']>1000000]
#df_selected['garajes'].unique()

In [ ]:
df_raw.loc[18045, 'url']

In [ ]:
df_selected.describe()

In [ ]:
#df_selected['m2_del_terreno'] > 1
#df_selected['gastos_comunes'] < 1e5
#df_selected['m2_edificados'] > 20

(
    df_selected.query('m2_del_terreno > 2')
    .query('gastos_comunes < 1e5')
    .query('m2_edificados > 20')
    #.pipe(lambda x: x.describe().transpose())
    .to_csv('../data/ready/num_dataset_houses.csv', index=False)
)
